In [1]:
from tensorflow.keras.layers import Input, SimpleRNN, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [2]:
# Things you should automatically know and have memorized
# N = number of samples
# T = sequence length
# D = numbers of input features
# M = number of hidden units
# K = number of output units

In [3]:
# Make some data
N = 1
T = 10
D = 3
K = 2
X = np.random.randn(N, T, D)
X

array([[[-1.06848285,  0.16501908,  2.28729801],
        [ 1.28274679,  1.08780526,  1.08745971],
        [-0.505869  , -0.71923571, -0.23249848],
        [ 2.72827354, -0.61360672,  2.08457672],
        [ 0.7597464 , -1.41360065, -0.52457993],
        [-0.4893981 ,  0.60407251, -1.49486313],
        [ 0.53023555, -0.85516845, -1.20979326],
        [ 2.04956946,  0.09749772,  2.1839952 ],
        [-0.03898986, -0.13467828, -0.74590031],
        [ 1.25577312, -0.95381906,  0.44013496]]])

In [5]:
# Make an RNN
M = 5 # number of hidden units
i = Input(shape=(T, D))
x = SimpleRNN(M)(i)
x = Dense(K)(x)

model = Model(i, x)

In [6]:
# Get the output
Yhat = model.predict(X)
print(Yhat)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
[[ 0.00640822 -0.34446296]]


In [7]:
# See if we can replace this output
# Get the weights first
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 10, 3)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ (None, 5)                   │              45 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 2)                   │              12 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 57 (228.00 B)

 Trainable params: 57 (228.00 B)

 Non-trainable params: 0 (0.00 B)

In [9]:
# See what's returned
model.layers[1].get_weights()

[array([[ 0.46833628,  0.41051692, -0.8474064 ,  0.84905344,  0.30710655],
        [-0.24550682,  0.45328885, -0.31665534,  0.71317285, -0.33319658],
        [-0.5549691 ,  0.1906001 , -0.47451034, -0.21399415,  0.01469475]],
       dtype=float32),
 array([[-0.7853904 , -0.12877437,  0.5194391 ,  0.12227295, -0.2860269 ],
        [ 0.06737661,  0.85768133,  0.20933734, -0.32666057, -0.33062655],
        [ 0.07825457, -0.13827199, -0.4402468 ,  0.23358966, -0.85227686],
        [-0.31564555,  0.47817522, -0.36965182,  0.6795785 ,  0.27064154],
        [-0.5223661 , -0.0054147 , -0.59657466, -0.60162354,  0.09618723]],
       dtype=float32),
 array([0., 0., 0., 0., 0.], dtype=float32)]

In [10]:
# check their shapes
# should make sense
# First output is input > hidden
# Second output is hidden > hidden
# Third output is bias term (vector of length M)
a, b, c = model.layers[1].get_weights()
print(a.shape, b.shape, c.shape)

(3, 5) (5, 5) (5,)


In [11]:
Wx, Wh, bh = model.layers[1].get_weights()
Wo, bo = model.layers[2].get_weights()

In [14]:
h_last = np.zeros(M) # initial hidden state
x = X[0] # the one and only sample
Yhats = [] # where we store the outputs

for t in range(T):
    h = np.tanh(x[t].dot(Wx) + h_last.dot(Wh) + bh)
    y = h.dot(Wo) + bo # we only care about this value on the last iteration
    Yhats.append(y)

    # important: assign h to h_last
    h_last = h

# pint the final output
print(Yhats[-1])

[ 0.00640837 -0.34446296]
